In [1]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("Enable GPU in your notebook settings.")


GPU is available!


In [2]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.0 MB/s eta 0:00:00:00:0100:01


In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load tokenizer and model
model_name = "Skywork/Skywork-o1-Open-Llama-3.1-8B"  # Example: Open-source LLaMA
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically load to GPU
    load_in_8bit=True   # Use 8-bit precision for memory efficiency
)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/906 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [21]:
# Input prompt
prompt = "What are the benefits of yoga?"


# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    inputs.input_ids,
    max_length=100,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True
)

# Decode the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What are the benefits of yoga? The benefits of yoga include reduced stress, improved flexibility, enhanced strength, and better overall health. Yoga is a low-impact exercise that can be done at home, making it convenient for busy schedules.

The benefits of yoga are:

1. Reduced stress: Yoga helps to reduce stress and anxiety by promoting relaxation and inner peace.
2. Improved flexibility: Yoga stretches and balances the body, increasing flexibility and range of motion.
3. Enhanced strength: Yoga builds muscle


In [22]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Factors and their significant choices
factors_with_choices = {
    "Health Condition": ["Good", "Bad", "Recovering", "Chronic Issues"],
    "Fitness Level": ["Beginner", "Intermediate", "Advanced"],
    "Age": ["Child", "Adult", "Senior"],
    "Goals and Needs": ["Relaxation", "Strength", "Flexibility", "Therapeutic Benefits"],
    "Lifestyle": ["Sedentary", "Moderately Active", "Highly Active"],
    "Pregnancy": ["Not Pregnant", "First Trimester", "Second Trimester", "Third Trimester"],
}

# Output widget to display selected choices
output = widgets.Output()

# Function to display selectors for all factors
def create_selectors():
    selectors = {}
    for factor, choices in factors_with_choices.items():
        selector = widgets.Dropdown(
            options=choices,
            value=choices[0],  # Default value
            description=f'{factor}:',
            style={'description_width': 'initial'},
        )
        selectors[factor] = selector
    return selectors

# Create dropdown selectors for all factors
selectors = create_selectors()

# Function to display the selected options
def show_selections(_):
    with output:
        output.clear_output()
        display(Markdown("### Selected Options"))
        for factor, selector in selectors.items():
            display(Markdown(f"**{factor}:** {selector.value}"))

# Create a button to confirm the selection
button = widgets.Button(description="Show Selections")
button.on_click(show_selections)

# Display all dropdown selectors
for factor, selector in selectors.items():
    display(selector)

# Display the button and output
display(button, output)


Dropdown(description='Health Condition:', options=('Good', 'Bad', 'Recovering', 'Chronic Issues'), style=Descr…

Dropdown(description='Fitness Level:', options=('Beginner', 'Intermediate', 'Advanced'), style=DescriptionStyl…

Dropdown(description='Age:', options=('Child', 'Adult', 'Senior'), style=DescriptionStyle(description_width='i…

Dropdown(description='Goals and Needs:', options=('Relaxation', 'Strength', 'Flexibility', 'Therapeutic Benefi…

Dropdown(description='Lifestyle:', options=('Sedentary', 'Moderately Active', 'Highly Active'), style=Descript…

Dropdown(description='Pregnancy:', options=('Not Pregnant', 'First Trimester', 'Second Trimester', 'Third Trim…

Button(description='Show Selections', style=ButtonStyle())

Output()

In [27]:
# Access selected options
selected_options = {factor: selector.value for factor, selector in selectors.items()}

# Display the selected options programmatically
print("Selected Options:")
for factor, value in selected_options.items():
    print(f"{factor}: {value}")


Selected Options:
Health Condition: Bad
Fitness Level: Advanced
Age: Senior
Goals and Needs: Relaxation
Lifestyle: Sedentary
Pregnancy: Not Pregnant


In [31]:
template = """
Health Condition: {health_condition}
Fitness Level: {fitness_level}
Age: {age}
Goals and Needs: {goals_and_needs}
Lifestyle: {lifestyle}
Pregnancy: {pregnancy}
"""

# Mapping the selected options to the template keys
filled_template = template.format(
    health_condition=selected_options.get("Health Condition", "Not Specified"),
    fitness_level=selected_options.get("Fitness Level", "Not Specified"),
    age=selected_options.get("Age", "Not Specified"),
    goals_and_needs=selected_options.get("Goals and Needs", "Not Specified"),
    lifestyle=selected_options.get("Lifestyle", "Not Specified"),
    pregnancy=selected_options.get("Pregnancy", "Not Specified"),
)

# Print the filled template
print(filled_template)



Health Condition: Bad
Fitness Level: Advanced
Age: Senior
Goals and Needs: Relaxation
Lifestyle: Sedentary
Pregnancy: Not Pregnant



In [32]:
prompt = f"""
Given the following factors and their values:
{filled_template}
Select the best yoga pose from the following options: ["downdog", "goddess", "plank", "tree", "warrior2"] based on these factors. Provide the selected pose only and nothing else.
"""
print(prompt)


Given the following factors and their values:

Health Condition: Bad
Fitness Level: Advanced
Age: Senior
Goals and Needs: Relaxation
Lifestyle: Sedentary
Pregnancy: Not Pregnant

Select the best yoga pose from the following options: ["downdog", "goddess", "plank", "tree", "warrior2"] based on these factors. Provide the selected pose only and nothing else.



In [33]:


# prompt="which health condition will be cured by goddess yoga position ?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    inputs.input_ids,
    max_length=200,
    temperature=0.7,
    top_k=50,
    top_p=0.95,
    do_sample=True
)

# Decode the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Given the following factors and their values:

Health Condition: Bad
Fitness Level: Advanced
Age: Senior
Goals and Needs: Relaxation
Lifestyle: Sedentary
Pregnancy: Not Pregnant

Select the best yoga pose from the following options: ["downdog", "goddess", "plank", "tree", "warrior2"] based on these factors. Provide the selected pose only and nothing else.
The best yoga pose for the given factors is "downdog". 

This is a simple task that requires an algorithmic approach to match the criteria to the yoga poses. Here's the suitability of each pose for the health condition, fitness level, and goals and needs:

- Downdog: suitable for all the given factors.
- Goddess: suitable for all the given factors.
- Plank: suitable for all the given factors.
- Tree: suitable to the given factors.
- Warrior2: suitable for all the given
